*All content and data presented in the articles on this platform are sourced from the comprehensive boxset titled “Market Risk Analysis” by Carol Alexander. The author of the articles acknowledges and respects the intellectual property rights and copyrights held by the original author of the boxset. The purpose of sharing this information is solely for educational and informational purposes, and no infringement of intellectual property rights is intended.*

In [72]:
from scipy import stats
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
import statsmodels.api as sm

import pathlib
import sys
utils_path = pathlib.Path().absolute().parent.parent
sys.path.append(utils_path.__str__())
import utils.layout as lay
from utils.functions import PCA

In [73]:
pio.templates.default = 'simple_white+blog_mra'

# Equally Weighted Averages

This section describes how to derive equally weighted estimates for volatility and correlation.

## Unconditional Variance and Volatility

The traditional approach for calculating volatility entails derving an unbiased estimate of the variance using an equally weighted average of squared returns and converting that value into a volatility estimate by applying the squared root of time rule. Assuming that each return has zero mean:

\begin{equation}
\hat{\sigma_{it}}^2 = T^{-1} \sum_{h=1}^{T} r_{i, t-k}^2
\end{equation}

After having obtained an unbiased estimate of the variance, we can use the square-root-of-time rule to convert it into volatility as:

\begin{equation}
Equally\ Weighted\ Volatility\ Estimate = \hat{\sigma_{it}} \sqrt{250}
\end{equation}

### *Example: Estimate of FTSE 100 volatility*

*Daily closing values on the FTSE 100 index between Friday 10 August and Friday 24 August
2007 are shown in the below table. Use these data to estimate the volatility of the FTSE 100
index at the close of the market on 24 August 2007.*

In [3]:
df = pd.read_excel(r"data/Examples_II.3.xls", sheet_name="EX_II.3.5&8").iloc[:11, :2].set_index("Date")
df.index = df.index.date
df["LogRet"] = np.log(df/df.shift())[1:]

$$\begin{array}{lrr}
\hline
 & FTSE & LogRet \\
\hline
2007-08-10 & 6038.30 & NaN \\
2007-08-13 & 6219.00 & 0.03 \\
2007-08-14 & 6143.50 & -0.01 \\
2007-08-15 & 6109.30 & -0.01 \\
2007-08-16 & 5858.90 & -0.04 \\
2007-08-17 & 6064.20 & 0.03 \\
2007-08-20 & 6078.70 & 0.00 \\
2007-08-21 & 6086.10 & 0.00 \\
2007-08-22 & 6196.00 & 0.02 \\
2007-08-23 & 6196.90 & 0.00 \\
2007-08-24 & 6220.10 & 0.00 \\
\hline
\end{array}
$$

### *(1) Zero mean assumption*

In [45]:
df["SqRet"] = df["LogRet"]**2
var_FTSE = df["SqRet"].mean()
vol_FTSE  = np.sqrt(var_FTSE*250)
print("Variance: {}".format(round(var_FTSE, 6)))
print("Volatility: {}".format(round(vol_FTSE, 3)))

Variance: 0.000433
Volatility: 0.329


The reason why we use log returns rather than ordinary returns are the following:
* at daily frequency there is very little difference from the two. For lower frequencies e.g. weekly, montly etc. it is conventional to use ordinary returns instead
* the geometric brownian motion assumes a price process whose log returns are normally distributed
* unlike ordinary returns, the h-period log return is the sum of h consecutive one-period returns.

If the expected return is assumed to be zero then the variance obtained in Equation (1) is an unbiased estimator i.e. $E(\hat{\sigma}^2) = \sigma^2$ and so $\sqrt{E(\hat{\sigma}^2)} = \sigma$

Otherwise, the expected return has to be calculated from the sample, and an unbiased estimate of the sample variance is derived as:

\begin{equation}
\hat{s_{it}}^2 = (T-1)^{-1} \sum_{h=1}^{T}(r_{i, t-k} -  \bar{r_{i}})^2
\end{equation}


### *(2) Without zero mean assumption*

In [46]:
avg_log_ret = df["LogRet"].mean()
df["SqMeanDev"] = (df["LogRet"] - avg_log_ret)**2

variance = sum(df["SqMeanDev"][1:])/(len(df["SqMeanDev"][1:]) - 1)
volatility = np.sqrt(variance*250)
print("Variance: {}".format(round(variance, 6)))
print("Volatility: {}".format(round(volatility, 3)))

Variance: 0.000471
Volatility: 0.343


The result is an equally weighted volatility estimate of 34.32%. This is quite different from
the volatility of 32.9% that we obtained in the previous example, based on the zero mean
return assumption, since the sample is small and there is a considerable sampling error. With
a large sample the difference would be much smaller.

## Unconditional Covariance and Correlation


Equally weighted estimate of covariance can be computed as:

\begin{equation}
\hat{\sigma_{ijt}} = T^{-1} \sum_{h=1}^{T} r_{i, t-k} r_{j, t-k}
\end{equation}

However, this only works for high frequency data and under the assumption of zero mean returns.
When these conditions are not met, it is recommended to:
* use ordinary returns instead of log returns
* take the sum of the cross prodcut of the mean deviations of returns
* use T-1 instead of T in the denominator

Once the two variances and the covariance metrics are calculated, a correlation esimate can be derived as:

\begin{equation}
Equally\ Weighted\ Correlation\ Estimate = \hat{\rho_{ijt}} = \frac{\hat{\sigma_{ijt}}}{\hat{\sigma_{it}}\hat{\sigma_{jt}}}
\end{equation}


### *Example: Estimate of FTSE-S&P correlation*

*Use the data in table below, which is taken over the same period as the FTSE 100 data, to
estimate the correlation between the FTSE 100 and S&P 500 index*

In [47]:
df1 = pd.read_excel(r"data/Examples_II.3.xls", sheet_name="EX_II.3.7").iloc[:11, :2].set_index("Date").astype(float)
df1.index = df1.index.date
df1["LogRet"] = np.log(df1/df1.shift())
df1["SqRet"] = df1["LogRet"]**2
df1["CrossProduct"] = df1["SqRet"]*df1["SqRet"]

In [64]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=(100*df["FTSE"]/df["FTSE"].values[0]),
                    mode='lines',
                    name='FTSE'))
fig.add_trace(go.Scatter(x=df1.index, y=(100*df1["SP500"]/df1["SP500"].values[0]),
                    mode='lines',
                    name='SP500'))
fig.update_layout(title_text = "FTSE100 and S&P500")
fig.show()

In [52]:
X = np.stack((df["LogRet"][1:], df1["LogRet"][1:]))
cov_mx =  np.cov(X )

$$ Covariance Matrix$$
$$\begin{bmatrix}
  0.00047 &  0.00013\\
  0.00013 &  0.00015
\end{bmatrix}$$

In case of equally weighted averages based on i.i.d. processes, we get the same results for both estimates and forecasts as the parameters are considered constant.
However, it is usual to take the horizon of the forecast equal to the frequency of the data used for the estimate.
Thus, daily returns data give a 1-day forecast.


# Precision of Equally Weighted Averages

In general, variance and volatility are not observable because, unlike financial asssets, they are not traded. They only exist in the context of the model used to estimate it. For this reason, we use confidence interval to gauge the accuracy of equally weighted estimates and add the ^ to denote that the number is an estimate of forecast.

### Confidence intervals for variance and volatility

If the assume T normally distributed returns with mean 0, then $T \hat{\sigma}^2 / \sigma^2 $ will have a chi-squared distribution with T degrees of freedom and the associated confidence interval for the variance $\sigma^2$ is

\begin{equation}
\left( \frac{T \hat{\sigma}^2}{\chi_{\alpha/2, T}^2},  \frac{T \hat{\sigma}^2}{\chi_{1 -\alpha/2, T}^2} \right) 
\end{equation}


Based on the consideration that the square root function is monotonic increasing and that percentiles are invariant under strictly monotonic inscreasing transfrormation such that

\begin{equation}
P(c_{1} < X< c_{u}) = P(f(c_{1}) < f(X) < f(c_{u}))
\end{equation}

we can simply obtain our confidence intervals for volatility by taking the sqare root and applying annualization.

### *Example: confidence interval for variance*

*Assuming the daily log returns on the FTSE 100 are normally distributed, use the sample
given in the example above to construct a 95% confidence interval for the variance of the returns.*

In [54]:
def calculate_variance_ci(var_FTSE, alpha, dfreed):
    prob1 = alpha/2
    prob2 = 1 - alpha/2
    cv1 = stats.chi2.ppf(prob1, dfreed)
    cv2 = stats.chi2.ppf(prob2,dfreed)
    up = T*var_FTSE / cv1
    lw = T*var_FTSE / cv2
    return up, lw

In [55]:
import scipy.stats as stats
T = len(df["LogRet"][1:])
alpha = 0.05
upper_bound, lower_bound = calculate_variance_ci(var_FTSE, alpha, T)

In [56]:
x = np.linspace(0, 25, 1000)  
pdf = stats.chi2.pdf(x, T)
fig = px.line(x=x, y=pdf)
fig.update_layout(title_text = "Chi-squared PDF (df=10)",  xaxis_title = "Chi-squared value", yaxis_title = "Probability Density")
fig.show()

In [57]:
print("For Variance")
print("Upper Bound: {}".format(round(upper_bound, 4)))
print("Lower Bound: {}".format(round(lower_bound,  5)))
print("-"*30)
print("For Volatility")
print("Upper Bound: {}".format(round(np.sqrt(upper_bound*250), 3)))
print("Lower Bound: {}".format(round(np.sqrt(lower_bound*250),  3)))

For Variance
Upper Bound: 0.0013
Lower Bound: 0.00021
------------------------------
For Volatility
Upper Bound: 0.577
Lower Bound: 0.23


In [67]:
est_var = 1
Ts = np.arange(10, 210, 10)
up_bounds =  [t*est_var/ stats.chi2.ppf(0.025, t) for t in Ts]
lw_bounds =  [t*est_var/stats.chi2.ppf(0.975, t) for t in Ts]
ci_bounds = pd.DataFrame(np.stack((up_bounds, lw_bounds)).T, index=Ts, columns=["Up", "Lw"])

In [68]:
fig = px.line(ci_bounds)
fig.update_layout(title_text="Confidence Interval for Variance Forecast", 
                              xaxis_title = "T", yaxis_title = "Estimated Variance",
                             showlegend=False,)
fig.show()

## Standard Error of Variance, Volatility and Correlation

**VARIANCE** 

A point estimate of volatility is the expectation of the distribution of the volatility estimator. To measure the accuracy of this point estimate we can use the *standard error of the estimator*, which is the standard deviation of its distribution.
To derive a formulation for the standard error we start assuming that $E(r_{t}) = 0$. Under this assumption the variance  of the variance estimate is:

\begin{equation}
V(\sigma_{t}^2) = T^{-2}\sum_{k=1}^{T} V(r_{t-k})^2
\end{equation}

If $X = r^2$ and considering that $V(X) = E(X^2) - E(X)^2$, we get that:

\begin{equation}
V(r_{t}^2) = E(r_{t}^4) - E(r_{t}^2)^2
\end{equation}

Since we have assumed $E(r_{t}) = 0$ and that returns are normally distributed, we obtain

$$
E(r_{t}^2) = \sigma^2 
\quad E(r_{t}^4) = 3\sigma^4
$$

which leads to

\begin{equation}
V(r_{t}^2) = 3\sigma^4 - \sigma^2 = 2\sigma^4
\end{equation}

Substituing Equation (8) into Equation (6) we get that the standard error and the percentage standard error of the variance are :

\begin{equation}
s.e.(\hat{\sigma}^2) = \sqrt{\frac{2}{t}} \sigma^2 \quad \frac{s.e.(\hat{\sigma}^2)}{\sigma^2} = \sqrt{\frac{2}{t}} 
\end{equation}

**VOLATILITY**

The standard error of the volatility is not simply the square root of the standard error of the variance
$$s.e.(\sigma) \neq \sqrt{s.e.(\sigma^2)} $$


**CORRELATION**

The correlation estimate divided by the standard error has a student $t$ distribution with T degrees of freedom:

\begin{equation}
V(\hat{\rho}) = T^{-1} (1 - \rho^2)
\end{equation}

\begin{equation}
\frac{\hat{\rho}\sqrt{T}}{\sqrt{1-\hat{\rho}^2}} \sim t_{T} 
\end{equation}

### Example: standard error of volatility

*An equally weighted volatility estimate is 20%, based on a sample of 100 observations.
Estimate the standard error of the estimator and find an interval for the estimate based on
one-standard-error bounds.*

In [69]:
T = 100
vol = 0.2
var = vol**2
# standard errors
se_vol = 1/np.sqrt(2*T)
se_var = np.sqrt(2/T)
#bounds
lb_vol = vol - vol*se_vol
ub_vol = vol + vol*se_vol
lb_var = var - vol*se_var
ub_var = var + vol*se_var

In [82]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

mean = 0.2  
std_dev = mean/np.sqrt(2*100) 
x = np.linspace(mean - 3 * std_dev, mean + 3 * std_dev, 1000)
pdf = norm.pdf(x, loc=mean, scale=std_dev)

# Plot Estimator
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=pdf, mode='lines', name=f'Mean={mean}, Std Dev={std_dev}'))
fig.add_vline(x=vol, line_width=3, line_dash="dash", line_color="green")
fig.add_vline(x=ub_vol, line_width=3, line_dash="dash", line_color="red")
fig.add_vline(x=lb_vol, line_width=3, line_dash="dash", line_color="red")

fig.update_layout(
    title='One Standard Error for Volatility',
    xaxis_title='',
    yaxis_title='PDF',
    legend=dict(x=0.7, y=0.9),
)
fig.show()

### Example: standard error of correlation

*A historical correlation estimate of 0.2 is obtained using 36 observations. Is this significantly
greater than 0?*

In [83]:
corr = 0.2
N = 36

def tstats_corr(corr, N):
    return (corr*np.sqrt(N))/np.sqrt(1- corr**2)

t  = tstats_corr(corr, N)
print(t)

1.2247448713915894


The 10% upper critical value of the $t$ distribution with 36 degrees of freedom is greater then this 1.225. Hence we cannot reject the null hypthesis of correlation greater than 0